In [ ]:
import sys
sys.path.append("..")

import random
import math
import time
from io import BytesIO
from pathlib import Path
from collections import OrderedDict
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Union

import PIL.Image
import PIL.ImageDraw

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from IPython.display import display

from src.datasets import *
from src.util.image import *
from src.util import *
from src.algo import *
from src.models.decoder import *
from src.models.transform import *

def resize(img, scale: float, mode: VF.InterpolationMode = VF.InterpolationMode.NEAREST):
    return VF.resize(img, [max(1, int(s * scale)) for s in img.shape[-2:]], mode, antialias=False)

def plot_samples(
        iterable, 
        total: int = 32, 
        nrow: int = 8, 
        return_image: bool = False, 
        show_compression_ratio: bool = False,
        label: Optional[Callable] = None,
):
    samples = []
    labels = []
    f = ImageFilter()
    try:
        for idx, entry in enumerate(tqdm(iterable, total=total)):
            image = entry
            if isinstance(entry, (list, tuple)):
                image = entry[0]
            if image.ndim == 4:
                image = image.squeeze(0)
            samples.append(image)
            if show_compression_ratio:
                labels.append(round(f.calc_compression_ratio(image), 3))
            elif label is not None:
                labels.append(label(entry) if callable(label) else idx)
                
            if len(samples) >= total:
                break
    except KeyboardInterrupt:
        pass
    
    if labels:
        image = VF.to_pil_image(make_grid_labeled(samples, nrow=nrow, labels=labels))
    else:
        image = VF.to_pil_image(make_grid(samples, nrow=nrow))
    if return_image:
        return image
    display(image)

In [ ]:
class DCGAN(nn.Module):
    def __init__(
            self,
            num_input: int = 100,
            channels: int = 128,
            channels_out: int = 3,
    ):
        super().__init__()
        self.num_input = num_input
        self.layers = nn.Sequential()
        self.layers.add_module("project", nn.Linear(self.num_input, channels * 8 * 4 * 4))
        self.layers.add_module("reshape", Reshape((channels * 8, 4, 4)))
        
        self.layers.add_module("conv1", nn.ConvTranspose2d(channels * 8, channels * 4, kernel_size=2, stride=2))
        self.layers.add_module("bn1", nn.BatchNorm2d(channels * 4))
        self.layers.add_module("act1", nn.LeakyReLU())
        
        self.layers.add_module("conv2", nn.ConvTranspose2d(channels * 4, channels * 2, kernel_size=2, stride=2))
        self.layers.add_module("bn2", nn.BatchNorm2d(channels * 2))
        self.layers.add_module("act2", nn.LeakyReLU())
        
        self.layers.add_module("conv3", nn.ConvTranspose2d(channels * 2, channels_out, kernel_size=2, stride=2))
        self.layers.add_module("act3", nn.Tanh())
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)


model = DCGAN()
print(f"params: {num_module_parameters(model):,}")
print(model(torch.randn(100)).shape)
model

In [ ]:
VF.to_pil_image(make_grid(model(torch.randn(32, 100))))


In [ ]:
nn.Softmax(dim=-1)(torch.Tensor([[-2, 3]]))